# Init

## Load necessary libraries. 
We suggest you use anaconda to setup a python 2 environment and install the following dependencies:
 * mdtraj https://anaconda.org/omnia/mdtraj
 * sklearn https://scikit-learn.org/stable/install.html
 * bio pandas https://anaconda.org/conda-forge/biopandas
 
Maybe I've missed some library. In that case please add it to the list

In [ ]:
from __future__ import absolute_import, division, print_function

import logging
import sys

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
    format='%(asctime)s %(name)s-%(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')
import os
import numpy as np
import mdtraj as md
from modules import utils, feature_extraction as fe, postprocessing, visualization
from modules.data_generation import DataGenerator
from modules import filtering, data_projection as dp
import matplotlib.pyplot as plt

logger = logging.getLogger("notebook")
working_dir = "/media/oliverfl/CHARMANDER/bachelor-thesis-2019/"  #TODO change directory
logger.info("Done with init")

## Load the MD trajectories
To use mdtraj, see http://mdtraj.org/latest/examples/introduction.html

There are currently **6** different simulations avaialable:
 * 3 simulations which have an agonist ligand (a drug favouring the active state) bound to the receptor. We call this __holo__
 
 
 * 3 simulations which have no ligand. We call this __apo__
 
 
 * 2 simulations (one holo; one apo) have the conserved residue Asp79 protonated (an extra proton). We call this __ash79__
 
 
 * 2 simulations (one holo; one apo) have Asp79 deprotonated but with a sodium ion forced next to it. We call this __asp79_Na__
 
 
 * 2 simulations (one holo; one apo; you get it now) have Asp79 deprotonated and no sodium force bound. We call this __asp79__. **Start with just these two!**
 
There is more data available in the directory 'longer'. If you need even more, tell me

In [ ]:
traj_dir = working_dir + "string_trajectories/shorter/" #TODO change directory
apo_asp_traj = md.load(traj_dir + "asp79-apo-swarms-nowater-nolipid.xtc", 
                        top=traj_dir + "asp79-apo-swarms-nowater-nolipid.pdb") 
holo_asp_traj = md.load(traj_dir + "asp79-holo-swarms-nowater-nolipid.xtc", 
                        top=traj_dir + "asp79-holo-swarms-nowater-nolipid.pdb")  
logger.info("Loaded trajectories with properties %s, %s", holo_asp_traj, apo_asp_traj)

### Optional: select a subset om the atoms in the trajectory
See http://mdtraj.org/latest/examples/atom-selection.html 

In [ ]:
#TODO

## Compute the interatomic distances for mdtrajectories
See http://mdtraj.org/1.6.2/api/generated/mdtraj.compute_contacts.html

In [ ]:
holo_asp_distances, holo_asp_residue_pairs = md.compute_contacts(holo_asp_traj,
                                   contacts="all",
                                   scheme="closest-heavy", #You may want to use 'ca'
                                   ignore_nonprotein=True)
apo_asp_distances, apo_asp_residue_pairs = md.compute_contacts(apo_asp_traj,
                                   contacts="all",
                                   scheme="closest-heavy", #You may want to use 'ca'
                                   ignore_nonprotein=True)

if holo_asp_distances.shape[1] != apo_asp_distances.shape[1]:
    raise Exception("Different number of contacts in the two simulations. Must be the same")
logger.info("Done computing %s and %s distances", holo_asp_distances.shape, apo_asp_distances.shape)

## Merge this together into a data format suitable for training - samples and labels

Once you're done you want to train classifiers to predict the labels with the samples as input. i.e. based on the contacts in the protein, predict if it was apo or holo etc

In [ ]:
samples = np.empty((len(holo_asp_distances) + len(apo_asp_distances), apo_asp_distances.shape[1]))
labels = np.empty((samples.shape[0],))
samples[0:len(holo_asp_distances)] = holo_asp_distances
samples[len(holo_asp_distances):] = apo_asp_distances
labels[0:len(holo_asp_distances)] = 1 #We label holo with '1' 
labels[len(holo_asp_distances):] = 2 # and apo with '2' 


### Map the indices of the samples to the right residue number

Make sure to understand the differnece between the residue index in the topology and the biological residue number

In [ ]:

feature_to_resids = np.empty((samples.shape[1], 2)) #This array tells us which residues the index of a certain feature correspond to. 
for feature_idx, (res1h, res2h) in enumerate(holo_asp_residue_pairs):
    res1a, res2a = apo_asp_residue_pairs[feature_idx]
    #Convert to biological residue number - this is independent of what your topology (which is a datastructure) looks like
    res1h = holo_asp_traj.top.residue(res1h).resSeq
    res2h = holo_asp_traj.top.residue(res2h).resSeq
    res1a = apo_asp_traj.top.residue(res1a).resSeq
    res2a = apo_asp_traj.top.residue(res2a).resSeq    
    if res1h != res1a or res2h != res2a:
        raise Exception("Features differ at index %s. Must be aligned. (%s!=%s or %s!=%s)", feature_idx, res1h, res1a, res2h, res2a)
    else:
        feature_to_resids[feature_idx, 0] = res1h
        feature_to_resids[feature_idx, 1] = res2h        
logger.info("Done. Created samples of shape %s", samples.shape)

### Optional: save these as numpy npy files.
Then you can load them again in the future and skip the first steps

You can also split this notebook into two separate script - one which computes the distances and one which does feature extraction

In [ ]:
#TODO samples.save("xxxxx.npy) etc

# Feature extraction

Start by looking at the following 3 methods for classification:

** Random Forest (RF)**
 * Background: https://en.wikipedia.org/wiki/Random_forest
 * Implementation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
 
** Multilayer Perceptron (MLP)** 
 * The most time consuming one (probably)
 *Background: https://scikit-learn.org/stable/modules/neural_networks_supervised.html
 * Implementation: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
 
** Kullback–Leibler (KL) Divergence **
 * see https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence
 

In [ ]:
n_iterations, n_splits = 1, 1 #Number of times to run and number of splits in cross validation
filter_by_distance_cutoff = False #Remove all distances greater than 0.5 nm (configurable limit). Typically residues close to each other contribute most to the stability of the protein
use_inverse_distances = True #Usually it is a good idea to take the inverse of the distances since a larg number then indicates two residues in contact -> stronger interaction

feature_extractors = [
     fe.MlpFeatureExtractor(samples, labels, n_splits=n_splits, n_iterations=n_iterations, 
                             hidden_layer_sizes=(100,), #You may need to tweak this
                             activation="relu", #use "relu" or logistic
                             randomize=True, # set to false for reproducability
                             filter_by_distance_cutoff=filter_by_distance_cutoff),                         
     fe.RandomForestFeatureExtractor(samples, labels, n_splits=n_splits, n_iterations=n_iterations,
                             filter_by_distance_cutoff=filter_by_distance_cutoff),
     fe.KLFeatureExtractor(samples, labels, n_splits=n_splits,
                            filter_by_distance_cutoff=filter_by_distance_cutoff),    
############ Unsupervised learning methods, skip them for a start#######################
#      fe.RbmFeatureExtractor(rbm_data, cluster_indices, n_splits=n_splits, n_iterations=n_iterations, 
#                           n_components=8,
#                            use_inverse_distances=use_inverse_distances,
#                           filter_by_distance_cutoff=filter_by_distance_cutoff),    
#      fe.PCAFeatureExtractor(data, cluster_indices, n_splits=n_splits,
#                            filter_by_distance_cutoff=filter_by_distance_cutoff),    
]
logger.info("Done. using %s feature extractors", len(feature_extractors))


## Run feature extraction - this may take a little while

Every feature extractor will contain the "raw" data, the relevance per feature. For KL and MLP you will also get the relevance for apo and holo separately. For MLP you can even get it per frame, but that requires some code change I think

In [ ]:
results = []
for extractor in feature_extractors:
    logger.info("Computing relevance for extractors %s", extractor.name)
    feature_importance, std_feature_importance, errors = extractor.extract_features()
    #logger.info("Get feature_importance and std of shapes %s, %s", feature_importance.shape, std_feature_importance.shape)
    results.append((extractor, feature_importance, std_feature_importance, errors))
logger.info("Done")

# Postprocessing

Again, you may want to save the relevance as npy files for faster access in the future

## Now use the computed relevance and postprocess them to something more useful

Once you get the hang of it you might want to write your own code for postprocessing and visualization to get the results you're looking for

In [ ]:
postprocessors = []
for (extractor, feature_importance, std_feature_importance, errors) in results:
    p = postprocessing.PostProcessor(extractor, feature_importance, std_feature_importance, errors, labels,
                                     working_dir  + "analysis/", 
                                     pdb_file=working_dir + "analysis/apo_only_protein.pdb", 
                                     feature_to_resids=feature_to_resids, 
                                     filter_results=True)
    p.average()
    p.evaluate_performance()
    p.persist()
    postprocessors.append([p])
logger.info("Done")


## Visualize the results

In [ ]:
visualization.visualize(postprocessors,
          show_importance=True, 
          show_performance=False, 
          show_projected_data=False)

logger.info("Done. The settings were n_iterations, n_splits = %s, %s.\nFiltering (filter_by_distance_cutoff = %s)", 
            n_iterations, n_splits, filter_by_distance_cutoff)

# Next steps
**1) Look at the generated structures and the trajectories** 
 * Spend some time on this and really get used to looking at 3D protein structures. The 'beta' value of each residue should be set to the relevance
 * Do these results make sense? I.e. the features picked up by the feature extractor, can you see how they change with visual inspection 
 
 
**2) Do more qualitative analysis** 
 * Read last year's bachelor thesis and look at what was done there 
 * What is the efffect of cross validation?
 * Do scatter plots for different features and see if you can easily make the different conditions cluster together. 
 
 
**3) Summarize work so far**
 * Write down the conclusions you've made so far
 * Save the plots and the protein structures for the report. 
 * If there are errors or it doesn't work at all, try and find the reason why
 
 
**4) Now go back and try to include more than just two conditions**
 * do multi class classfication, try and separate simulations with deprotonated Asp79 from neutrally charged Asp79 and so on ... 
 * You might not have time to look at everything so think carefully about what the most relevant/interesting questions to ask are based on your results so far. 
 * Iterate through the steps here until you get bored
 
**Optional work**
 * Try the methods for unsupervised learning, PCA and RBM specifically
 * Try even more methods for dimensionality reduction and feature extraction. There is a lot available in scikit learn
 * Use other types of features (dihedral angles, raw xyz coordinates, contacts with different cutoffs etc)  and compare
 